In [1]:
import csv, json, os, re, shutil
import subprocess, sys, threading, traceback, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

Downloaded Allegheny_County_Voting_District_Boundaries_Fall_2016__present.zip from https://data.wprdc.org/dataset/allegheny-county-voting-district-boundaries-fall-2016-present


In [ ]:
!unzip Allegheny_County_Voting_District_Boundaries_Fall_2016__present.zip

In [2]:
!gdalsrsinfo Allegheny_County_Voting_District_Boundaries_Fall_2016__present.prj


PROJ.4 : '+proj=longlat +datum=WGS84 +no_defs '

OGC WKT :
GEOGCS["GCS_WGS_1984",
    DATUM["WGS_1984",
        SPHEROID["WGS_84",6378137,298.257223563]],
    PRIMEM["Greenwich",0],
    UNIT["Degree",0.017453292519943295]]



In [3]:
!shp2pgsql \
 Allegheny_County_Voting_District_Boundaries_Fall_2016__present.shp \
 allegheny_county_precincts_2016 > create_acvd_2016


Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]


In [4]:
!head create_acvd_2016

SET CLIENT_ENCODING TO UTF8;
SET STANDARD_CONFORMING_STRINGS TO ON;
BEGIN;
CREATE TABLE "allegheny_county_precincts_2016" (gid serial,
"fid" numeric(10,0),
"objectid_1" numeric(10,0),
"district_1" numeric(10,0),
"ward_1" numeric(10,0),
"municode_1" numeric(10,0),
"yrapprov_1" numeric(10,0),


In [9]:
!psql -q -d census2010 < create_acvd_2016

                              addgeometrycolumn                               
------------------------------------------------------------------------------
 public.allegheny_county_precincts_2016.geom SRID:0 TYPE:MULTIPOLYGON DIMS:2 
(1 row)



In [10]:
psql('\d allegheny_county_precincts_2016')

\d allegheny_county_precincts_2016
Finished execution in 0.168767 secs:
Table "public.allegheny_county_precincts_2016"
   Column   |          Type          |                                   Modifiers                                   
------------+------------------------+-------------------------------------------------------------------------------
 gid        | integer                | not null default nextval('allegheny_county_precincts_2016_gid_seq'::regclass)
 fid        | numeric(10,0)          | 
 objectid_1 | numeric(10,0)          | 
 district_1 | numeric(10,0)          | 
 ward_1     | numeric(10,0)          | 
 municode_1 | numeric(10,0)          | 
 yrapprov_1 | numeric(10,0)          | 
 mwd_nopa_1 | character varying(80)  | 
 opa_muni_1 | numeric(10,0)          | 
 mwd_pad_1  | character varying(80)  | 
 pseud4_12  | character varying(80)  | 
 pseudonu_5 | numeric(10,0)          | 
 muni_war_1 | character varying(80)  | 
 geom       | geometry(MultiPolygon) | 
Indexes:

In [13]:
psql('SELECT fid,objectid_1,district_1,ward_1,municode_1,yrapprov_1,mwd_nopa_1,opa_muni_1,mwd_pad_1,pseud4_12,muni_war_1 FROM allegheny_county_precincts_2016 LIMIT 10')

SELECT fid,objectid_1,district_1,ward_1,municode_1,yrapprov_1,mwd_nopa_1,opa_muni_1,mwd_pad_1,pseud4_12,muni_war_1 FROM allegheny_county_precincts_2016 LIMIT 10
Finished execution in 0.108614 secs:
fid  | objectid_1 | district_1 | ward_1 | municode_1 | yrapprov_1 | mwd_nopa_1 | opa_muni_1 | mwd_pad_1 | pseud4_12 |        muni_war_1         
------+------------+------------+--------+------------+------------+------------+------------+-----------+-----------+---------------------------
 1001 |       1004 |          4 |      0 |        104 |       1994 | 10404      |        877 | 1040004   | 0014      | BALDWIN BR DIST 4
 1002 |       1005 |          7 |     29 |        188 |       1988 | 188297     |        100 | 18802907  | 0961      | PITTSBURGH WARD 29 DIST 7
 1003 |       1006 |          1 |      8 |        198 |       1993 | 19881      |        941 | 1980801   | 1094      | SCOTT WARD 8 DIST 1
 1004 |       1007 |          2 |      0 |        132 |          0 | 13202      |        8

Next steps:

Download Results by Precinct from 2016 election, like so:

https://data.wprdc.org/dataset/election-results/resource/3e8b23b7-aa71-4850-9379-bce87f73d684

Either SQL JOIN, or python, to connect each precinct name to a maximum population, which could be total # of reg voters, or could be all the number of votes for all pres candidates put together.

For each precinct, compute MAXPOP random dots.


tiger2010_census2000_blocks and tiger2010_census2010_blocks
----------------------------------

In [ ]:
psql('\d tiger2010_census2000_blocks')
psql('\d tiger2010_census2010_blocks')

In [ ]:
psql('SELECT COUNT(*) FROM tiger2010_census2000_blocks')
psql('SELECT COUNT(*) FROM tiger2010_census2010_blocks')

In [ ]:
!pg_dump -t tiger2010_census2010_blocks timelapse | psql census2010

In [ ]:
psql("SELECT SUM(ST_AREA(geography(geom))) FROM tiger2010_census2000_blocks")
psql("SELECT SUM(ST_AREA(geography(geom))) FROM tiger2010_census2010_blocks")

In [14]:
!grep createuser *

00 Import 2000 Census Block Shapefiles.ipynb:    "    sudo -u postgres createuser --superuser $USER\n",
01 Import 2000-2010 Census Block Shapefiles.ipynb:    "    sudo -u postgres createuser --superuser $USER\n",
grep: capture: Is a directory
grep: columncache: Is a directory
grep: od-JT01-2011-home-tiles: Is a directory
grep: prototiles: Is a directory
grep: prototiles2: Is a directory
